In [ ]:
from matplotlib import pyplot
import pandas as pd
import numpy as np

## **Parte 1: Ecuación del calor**

La función debe devolver una matriz cuya entrada en la posición (i, j) represente la aproximación
numérica de la solución en el instante de tiempo t = i · ∆t y en la posición espacial x = j · ∆x.

### **Método Explicíto**

In [ ]:
#COMPLETAR

### **Método Implicíto**

In [ ]:
#COMPLETAR

#### Evaluación de las funciones con los parámetros: α = 1, ∆t = 0,001, ∆x = 0,05, T = 0,1

In [ ]:
#COMPLETAR

GIF - Con los valores devueltos en el caso anterior realizar un GIF de como al variar el instante de tiempo evolucionan los valores en cada lugar espacial. Incluir un pequeño análisis sobre el comportamiento de los métodos según los diferentes parámetros.


In [ ]:
#COMPLETAR

## **Parte 2: Ecuación de transporte**


### **Método Explicíto**

In [ ]:
#COMPLETAR

### **Método Implicíto**

In [ ]:
#COMPLETAR

#### Evaluación de las funciones con los parámetros: α = 1, ∆t = 0,001, ∆x = 0,05, T = 0,1

In [ ]:
#COMPLETAR

GIF - Con los valores devueltos en el caso anterior realizar un GIF de como al variar el instante de tiempo evolucionan los valores en cada lugar espacial. Incluir un pequeño análisis sobre el comportamiento de los métodos según los diferentes parámetros.


In [ ]:
#COMPLETAR